In [4]:
from typing import TypedDict, NotRequired
from langgraph.graph import StateGraph, START, END

In [5]:
class AgentState(TypedDict):
    number1: int
    operation: str
    number2: int
    number3: int
    operation2: str
    number4: int
    finalNumber: NotRequired[int]
    finalNumber2: NotRequired[int]

In [6]:
def router_node(state: AgentState) -> str:
    if state['operation'] == "+":
        return "addition_operation"
    elif state['operation'] == "-":
        return "subtraction_operation"
    else:
        raise ValueError(f'Invalid operation provided: {state["operation"]}')
    
    
def add_node(state: AgentState) -> AgentState:
    return {
        **state,
        "finalNumber": state["number1"] + state["number2"]
    }
    
def subtract_node(state: AgentState) -> AgentState:
    return {
        **state,
        "finalNumber": state["number1"] - state["number2"]
    }
    



In [7]:
def router_node2(state: AgentState) -> str:
    if state['operation2'] == "+":
        return "addition_operation2"
    elif state['operation2'] == "-":
        return "subtraction_operation2"
    else:
        raise ValueError(f'Invalid operation2 provided: {state["operation2"]}')
    
def add_node2(state: AgentState) -> AgentState:
    return {
        **state,
        "finalNumber2": state["number3"] + state["number4"]
    }
    
def subtract_node2(state: AgentState) -> AgentState:
    return {
        **state,
        "finalNumber2": state["number3"] - state["number4"]
    }

In [8]:
graph = StateGraph(AgentState)
graph.add_node("router", lambda state: state)
graph.add_edge(START, "router")


graph.add_node("add_node", add_node)
graph.add_node("subtract_node", subtract_node)


graph.add_conditional_edges(
    "router",
    router_node,
    {
        "addition_operation": "add_node",
        "subtraction_operation": "subtract_node"
    }
)

graph.add_node("router2", lambda state: state)
graph.add_edge("add_node", "router2")
graph.add_edge("subtract_node", "router2")


graph.add_node("add_node2", add_node2)
graph.add_node("subtract_node2", subtract_node2)


graph.add_conditional_edges(
    "router2",
    router_node2,
    {
        "addition_operation2": "add_node2",
        "subtract_node2": "subtract_node2"
    }
)

graph.add_edge("add_node2", END)
graph.add_edge("subtract_node2", END)



In [9]:
app = graph.compile()

result = app.invoke({
    "number1": 10,
    "operation": "-",
    "number2": 5,
    "number3": 7,
    "number4": 2,
    "operation2": "+"
})

print(result)

{'number1': 10, 'operation': '-', 'number2': 5, 'number3': 7, 'operation2': '+', 'number4': 2, 'finalNumber': 5, 'finalNumber2': 9}
